In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob

from tabulate import tabulate
printtab = lambda x : print(tabulate(x, headers='firstrow'))

Need to get the prevalences of the smoking groups in the years 2016 (start), 2021, 2026, 2031, 2051


With 95% confidence intervals

In [2]:
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2022-12-16_15-55-24-478281"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-03-29_14-16-04-931491"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-03-29_14-16-04-931491_updatedSep2023"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-10-11_23-47-30-192455"
base_dir = "/Users/nick/Documents/Gillings_work/uncertainty_analysis_data/uncertainty_analysis_2023-10-24_21-00-37-093241"
output_dir = os.path.join(base_dir, "outputs")
outputs_dirs = [os.path.join(output_dir, f"option_{i}") for i in range(6)]

In [3]:
collection_list_options = []
SQ_2031_rates = [] # status quo 2031 rates in separate list for comparison

for opt in range(6):
    outputs = outputs_dirs[opt]
    collection_list = []

    # for each arr, store a 2D array in the list
    # axis = 0 are the groups: menthol, nonmenthol, smoker, ecig/dual, former, never (3, 4, 3+4, 5, 2, 1)
    # axis = 1 are the years 2016, 2021, 2026, 2031, 2051
    for i,f in enumerate(sorted(glob(outputs + "/*.npy"))):
        arr = np.load(f)
        arr = arr[:,:,:,0,:] # age-restrict 18-64
        arr = arr[[0, 5, 10, 15, 35]] # get the years we are interested in
        arr = np.sum(arr, axis=(1,2)) # dont care about demographics
        arr = arr[:,:-1] # don't need dead people
        sums = np.sum(arr, axis=1) # total count for each year
        arr = arr / sums[:,np.newaxis] # get proportions
        arr = arr.T # transpose so we have (smoking groups, years) as axes
        arr = np.concatenate([ # want to add the smokers together too
            arr[0:4],
            (arr[2] + arr[3])[np.newaxis, :],
            arr[4][np.newaxis, :],
        ], axis=0)
        arr = arr[[2,3,4,5,1,0]] # re-order the smoking groups
        # add change 2016-2031 column and change from SQ column
        if opt == 0:
            SQ_2031_rates.append(arr[:,3]) # store the 2031 rates since we are looking at SQ now
            arr = np.concatenate([
                arr[:,:4], # 2016, 2021, 2026, 2031
                (arr[:,3] - arr[:,0])[:,np.newaxis], # change 2031 - 2016
                np.zeros((len(arr), 1)), # change from SQ (zeros)
                arr[:,4][:,np.newaxis], #2051
            ],axis=1)
        else:
            arr = np.concatenate([
                arr[:,:4], # 2016, 2021, 2026, 2031
                (arr[:,3] - arr[:,0])[:,np.newaxis], # change 2031 - 2016
                ((arr[:,3] - SQ_2031_rates[i]) / SQ_2031_rates[i])[:,np.newaxis], # change from SQ
                arr[:,4][:,np.newaxis], #2051
            ],axis=1)
        collection_list.append(arr)

    collection_list_options.append(collection_list)


        


In [4]:
status_quo_prevalences = []
for opt in range(6):
    # analyze collection_list and get 95% confidence intervals
    collection_list = np.array(collection_list_options[opt])

    mean_results = np.zeros_like(collection_list[0])
    upper_bound = np.zeros_like(collection_list[0])
    lower_bound = np.zeros_like(collection_list[0])

    for i in range(collection_list.shape[1]):
        for j in range(collection_list.shape[2]):
            mean = np.mean(collection_list[:,i,j])
            upper = np.percentile(collection_list[:,i,j], 97.5)
            lower = np.percentile(collection_list[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower

    # do a table

    mean_results = np.around(mean_results * 100, decimals=1)
    upper_bound = np.around(upper_bound * 100, decimals=1)
    lower_bound = np.around(lower_bound * 100, decimals=1)

    # change_in_prevalence = np.around(change_in_prevalence * 100, decimals=1)

    header = ["", "2016", "2021", "2026", "2031", "Change 2016-2031", "% Change from SQ 2031", "2051"]
    r1 = ["menthol"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[0], upper_bound[0], lower_bound[0]
    )]
    r2 = ["nonmenthol"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[1], upper_bound[1], lower_bound[1]
    )]
    r3 = ["menthol+nonmenthol"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[2], upper_bound[2], lower_bound[2]
    )]
    r4 = ["ecig/dual"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[3], upper_bound[3], lower_bound[3]
    )]
    r5 = ["former"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[4], upper_bound[4], lower_bound[4]
    )]
    r6 = ["nonsmoker"] + [f"{mean}%, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[5], upper_bound[5], lower_bound[5]
    )]

    rows = [r1, r2, r3, r4, r5, r6]

    tab = [header] + rows
    print(" ")
    print(" ")
    print(f"Smoking Rates, Ban Scenario #{opt}, with 95% Confidence Intervals")
    if opt == 0: print("** Status Quo Scenario **")
    printtab(tab)


 
 
Smoking Rates, Ban Scenario #0, with 95% Confidence Intervals
** Status Quo Scenario **
                    2016                 2021                 2026                 2031                 Change 2016-2031     % Change from SQ 2031    2051
------------------  -------------------  -------------------  -------------------  -------------------  -------------------  -----------------------  -------------------
menthol             5.8%, (5.7, 5.8)     6.0%, (5.8, 6.2)     5.6%, (5.4, 5.8)     5.3%, (5.1, 5.5)     -0.5%, (-0.7, -0.2)  0.0%, (0.0, 0.0)         4.8%, (4.6, 4.9)
nonmenthol          9.4%, (9.3, 9.4)     5.8%, (5.6, 6.1)     4.6%, (4.4, 4.9)     4.3%, (4.0, 4.5)     -5.1%, (-5.4, -4.9)  0.0%, (0.0, 0.0)         3.9%, (3.7, 4.1)
menthol+nonmenthol  15.1%, (15.1, 15.1)  11.9%, (11.6, 12.1)  10.2%, (9.9, 10.5)   9.5%, (9.2, 9.9)     -5.6%, (-5.9, -5.3)  0.0%, (0.0, 0.0)         8.7%, (8.5, 8.9)
ecig/dual           3.7%, (3.7, 3.7)     1.9%, (1.8, 2.0)     1.7%, (1.6, 1.8)    

# Now get the same table in an easy CSV format

In [5]:
print("CSV values to be read into software")
print("All values in this table are percentages")
print("There are 5 ban scenarios each with a table here")
print("In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)")
print("SQ = Status Quo scenario")
print("All numbers are rounded to 5 decimal places which should be more than we want for the final table")
status_quo_prevalences = []
for opt in range(6):
    # analyze collection_list and get 95% confidence intervals
    collection_list = np.array(collection_list_options[opt])

    mean_results = np.zeros_like(collection_list[0])
    upper_bound = np.zeros_like(collection_list[0])
    lower_bound = np.zeros_like(collection_list[0])

    for i in range(collection_list.shape[1]):
        for j in range(collection_list.shape[2]):
            mean = np.mean(collection_list[:,i,j])
            upper = np.percentile(collection_list[:,i,j], 97.5)
            lower = np.percentile(collection_list[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower


    # do a table

    mean_results = np.around(mean_results * 100, decimals=5)
    upper_bound = np.around(upper_bound * 100, decimals=5)
    lower_bound = np.around(lower_bound * 100, decimals=5)

    # change_in_prevalence = np.around(change_in_prevalence * 100, decimals=5)

    header = ["2016", "2021", "2026", "2031", "%Change2016-2031", "%ChangeFromSQ2031", "2051"]
    new_header = ["group,"]
    for e in header:
        new_header.append(e + "M,")
        new_header.append(e + "LB,")
        new_header.append(e + "UB,") 
    
    r1 = ["menthol,"]
    r2 = ["nonmenthol,"]
    r3 = ["menthol+nonmenthol,"]
    r4 = ["ecig/dual,"]
    r5 = ["former,"]
    r6 = ["neversmoker,"]

    rows = [r1, r2, r3, r4, r5, r6]
    
    
    for i, r in enumerate(rows):
        for m, ub, lb in zip(mean_results[i], upper_bound[i], lower_bound[i]):
            rows[i] += [f"{m},", f"{lb},", f"{ub},"]

    tab = [new_header] + rows
    print(" ")
    print(" ")
    print(f"Smoking Rates, Ban Scenario #{opt}, with 95% Confidence Intervals")
    if opt == 0: print("** Status Quo Scenario **")
    print(tabulate(tab))

CSV values to be read into software
All values in this table are percentages
There are 5 ban scenarios each with a table here
In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)
SQ = Status Quo scenario
All numbers are rounded to 5 decimal places which should be more than we want for the final table
 
 
Smoking Rates, Ban Scenario #0, with 95% Confidence Intervals
** Status Quo Scenario **
-------------------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ------------------  -------------------  -------------------  -------------------  --------------------  --------------------  ---------  ---------  ---------
group,               2016M,     2016LB,    2016UB,    2021M,     2021LB,    2021UB,    2026M,     2026LB,    2026UB,    2031M,     2031LB,    2031UB,    %Change2016-2031M,  %Change2016-2031LB,  %Change2016-2031UB,  %ChangeFromSQ2031M,  %ChangeFromSQ2031LB, 

# Now get mortality results!

Want to get mortality in the years 2016, 2021, 2026, 2031, 2051

with percent change between 2031 and 2016

for full population, non-Hispanic Black, poverty, not poverty

In [17]:

collection_list_options = []
total_pops_opt_group = []
SQ_mortality_2021_2031 = []

for opt in range(6):
    outputs = outputs_dirs[opt]
    collection_list = []
    this_total_pops = []

    # for each arr, store a 2D array in the list
    # axis = 0 are the groups: full, black, pov, not pov
    # axis = 1 are the years 2016, 2021, 2026, 2031, 2051
    # original dimensions are (year, black, pov, smoking group)
    # now, original dimensions are (year, black, pov, smoking group) # after we decided to add 65+ year olds separately
    for i,f in enumerate(sorted(glob(outputs + "/*.npy"))):
        arr = np.load(f)
        # arr = arr.sum(axis=3)
        arr = arr[:,:,:,0,:]

        # set the number of dead people in the first year (2016) to zero
        # and adjust all mortality afterward accordingly
        arr[:,:,:,5] -= arr[0,:,:,5].reshape((-1,2,2))

        total_pop = np.sum(arr[-1])
        total_black = np.sum(arr[-1,1,:,:])
        total_pov = np.sum(arr[-1,:,1,:])
        total_nonpov = np.sum(arr[-1,:,0,:])

        this_total_pops.append([total_pop, total_black, total_pov, total_nonpov])

        arr = arr[[0, 5, 10, 15, 35]] # get the years we are interested in
        arr = arr [:, :, :, 5] # only care about dead people

        arr = np.concatenate([
            (np.sum(arr, axis=(1,2)))[:, np.newaxis], # full pop
            (np.sum(arr[:,1,:], axis=1))[:, np.newaxis], # black
            (np.sum(arr[:,:,1], axis=1))[:, np.newaxis], # pov
            (np.sum(arr[:,:,0], axis=1))[:, np.newaxis], # not pov
        ], axis=1)

        arr = arr.T # now the dims are group, year

        # add the change from SQ column
        if opt == 0:
            SQ_mortality_2021_2031.append(arr[:,4] - arr[:,1])
            arr = np.concatenate([
                arr,
                np.zeros((len(arr), 1))
            ], axis=1)
        else:
            arr = np.concatenate([
                arr,
                ((arr[:,4] - arr[:,1]) - SQ_mortality_2021_2031[i])[:, np.newaxis],
            ], axis=1)
        
        print(arr.shape)
        assert False

        collection_list.append(arr)
    
    collection_list_options.append(collection_list)

    this_total_pops = np.array(this_total_pops)
    total_pops_opt_group.append(np.mean(this_total_pops, axis=0))

total_pops_opt_group = np.array(total_pops_opt_group)

(4, 6)


AssertionError: 

In [15]:
print(total_pops_opt_group)
print(total_pops_opt_group.shape)

backup = np.copy(total_pops_opt_group)

[[1.97598002e+08 2.45525890e+07 2.22443810e+07 1.75353621e+08]
 [1.97367337e+08 2.44572917e+07 2.21210353e+07 1.75246302e+08]
 [1.97325646e+08 2.44323987e+07 2.20595142e+07 1.75266132e+08]
 [1.97476839e+08 2.44891161e+07 2.21611624e+07 1.75315677e+08]
 [1.97391693e+08 2.44910410e+07 2.21594007e+07 1.75232293e+08]
 [1.97406181e+08 2.44909041e+07 2.21389877e+07 1.75267194e+08]]
(6, 4)


In [16]:
SQ_2051_mort = None
for opt in range(6):
    
    # analyze collection_list and get 95% confidence intervals
    collection_list = np.array(collection_list_options[opt])

    mean_results = np.zeros_like(collection_list[0])
    upper_bound = np.zeros_like(collection_list[0])
    lower_bound = np.zeros_like(collection_list[0])

    for i in range(collection_list.shape[1]):
        for j in range(collection_list.shape[2]):
            mean = np.mean(collection_list[:,i,j])
            upper = np.percentile(collection_list[:,i,j], 97.5)
            lower = np.percentile(collection_list[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower

    # # compute reduction in cumulative mortality
    # change_cummort = None
    # if opt == 0:
    #     SQ_2051_mort = np.concatenate([
    #         mean_results[:,-1][:,np.newaxis],
    #         upper_bound[:,-1][:,np.newaxis],
    #         lower_bound[:,-1][:,np.newaxis],
    #     ], axis=1)
    #     change_cummort = np.zeros_like(SQ_2051_mort)
    # else:
    #     change_cummort = np.concatenate([
    #         (mean_results[:,-1] - SQ_2051_mort[:,0])[:,np.newaxis],
    #         (upper_bound[:,-1] - SQ_2051_mort[:,2])[:,np.newaxis],
    #         (lower_bound[:,-1] - SQ_2051_mort[:,1])[:,np.newaxis],
    #     ], axis=1)

    # do a table

    mean_results = np.around(mean_results / 100000, decimals=1)
    upper_bound = np.around(upper_bound / 100000, decimals=1)
    lower_bound = np.around(lower_bound / 100000, decimals=1)
    # change_cummort = np.around(change_cummort / 100000, decimals=1)
    total_pops = np.around(total_pops_opt_group[opt] / 100000, decimals=1)

    header = ["", "2016", "2021", "2026", "2031", "2051", "Change from SQ 2021-2051", "total living"]
    r1 = ["full pop"] + [f"{mean}, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[0], upper_bound[0], lower_bound[0]
    )]
    r2 = ["black NH"] + [f"{mean}, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[1], upper_bound[1], lower_bound[1]
    )]
    r3 = ["poverty"] + [f"{mean}, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[2], upper_bound[2], lower_bound[2]
    )]
    r4 = ["not poverty"] + [f"{mean}, ({lower_bound}, {upper_bound})" for mean, upper_bound, lower_bound in zip(
        mean_results[3], upper_bound[3], lower_bound[3]
    )]

    rows = [r1, r2, r3, r4]

    # put in total pop
    for i, r in enumerate(rows):
        rows[i] += [f"{total_pops[i]}"]

    tab = [header] + rows
    print("")
    print("")
    print(f"Cumulative Mortality (units of 100,000), Ban Scenario #{opt}, with 95% Confidence Intervals")
    if opt == 0: print("** Status Quo Scenario **")
    printtab(tab)



Cumulative Mortality (units of 100,000), Ban Scenario #0, with 95% Confidence Intervals
** Status Quo Scenario **
             2016             2021                2026                2031                2051                   Change from SQ 2021-2051      total living
-----------  ---------------  ------------------  ------------------  ------------------  ---------------------  --------------------------  --------------
full pop     0.0, (0.0, 0.0)  29.6, (25.7, 33.5)  59.1, (53.8, 64.7)  87.6, (79.6, 93.6)  201.3, (193.0, 208.9)  0.0, (0.0, 0.0)                     1976
black NH     0.0, (0.0, 0.0)  3.7, (2.5, 5.0)     7.3, (5.7, 9.3)     10.6, (8.6, 13.2)   24.6, (21.9, 27.7)     0.0, (0.0, 0.0)                      245.5
poverty      0.0, (0.0, 0.0)  6.6, (5.4, 8.0)     13.6, (11.3, 15.7)  20.7, (18.0, 23.6)  51.8, (48.2, 55.9)     0.0, (0.0, 0.0)                      222.4
not poverty  0.0, (0.0, 0.0)  23.0, (19.6, 26.9)  45.5, (40.6, 50.8)  66.9, (60.8, 72.7)  149.5, (141.5, 1

# Mortality Data in CSV form

In [ ]:
print("CSV values to be read into software")
print("All values in this table are absolute counts")
print("There are 5 ban scenarios each with a table here")
print("In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)")
print("SQ = Status Quo scenario")
print("All numbers are rounded to 10 decimal places which should be more than we want for the final table")
SQ_2051_mort = None
for opt in range(6):
    
    # analyze collection_list and get 95% confidence intervals
    collection_list = np.array(collection_list_options[opt])

    mean_results = np.zeros_like(collection_list[0])
    upper_bound = np.zeros_like(collection_list[0])
    lower_bound = np.zeros_like(collection_list[0])

    for i in range(collection_list.shape[1]):
        for j in range(collection_list.shape[2]):
            mean = np.mean(collection_list[:,i,j])
            upper = np.percentile(collection_list[:,i,j], 97.5)
            lower = np.percentile(collection_list[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower

    # # compute reduction in cumulative mortality
    # change_cummort = None
    # if opt == 0:
    #     SQ_2051_mort = np.concatenate([
    #         mean_results[:,-1][:,np.newaxis],
    #         upper_bound[:,-1][:,np.newaxis],
    #         lower_bound[:,-1][:,np.newaxis],
    #     ], axis=1)
    #     change_cummort = np.zeros_like(SQ_2051_mort)
    # else:
    #     change_cummort = np.concatenate([
    #         (mean_results[:,-1] - SQ_2051_mort[:,0])[:,np.newaxis],
    #         (upper_bound[:,-1] - SQ_2051_mort[:,2])[:,np.newaxis],
    #         (lower_bound[:,-1] - SQ_2051_mort[:,1])[:,np.newaxis],
    #     ], axis=1)

    # do a table

    mean_results = np.around(mean_results / 100000, decimals=10)
    upper_bound = np.around(upper_bound / 100000, decimals=10)
    lower_bound = np.around(lower_bound / 100000, decimals=10)
    # change_cummort = np.around(change_cummort / 100000, decimals=10)
    total_pops = np.around(total_pops_opt_group[opt] / 100000, decimals=10)

    header = ["2016", "2021", "2026", "2031", "2051", "ChangeFromSQ2021-2051"]
    new_header = ["group,"]
    for e in header:
        new_header.append(e + "M,")
        new_header.append(e + "LB,")
        new_header.append(e + "UB,") 
    new_header += ["totalLiving"]

    r1 = ["fullPop,"]
    r2 = ["BlackNH,"]
    r3 = ["Poverty,"]
    r4 = ["NotPoverty,"]

    rows = [r1, r2, r3, r4]

    for i, r in enumerate(rows):
        for m, ub, lb in zip(mean_results[i], upper_bound[i], lower_bound[i]):
            rows[i] += [f"{m},", f"{lb},", f"{ub},", ]

    # put in total pop
    for i, r in enumerate(rows):
        # rows[i] += [f"{change_cummort[i,0]},", f"{change_cummort[i,2]},",f"{change_cummort[i,1]},"]
        rows[i] += [f"{total_pops[i]},"]

    tab = [new_header] + rows
    print("")
    print("")
    print(f"Cumulative Mortality (units of 100,000), Ban Scenario #{opt}, with 95% Confidence Intervals")
    if opt == 0: print("** Status Quo Scenario **")
    print(tabulate(tab))

CSV values to be read into software
All values in this table are absolute counts
There are 5 ban scenarios each with a table here
In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)
SQ = Status Quo scenario
All numbers are rounded to 10 decimal places which should be more than we want for the final table


Cumulative Mortality (units of 100,000), Ban Scenario #0, with 95% Confidence Intervals
** Status Quo Scenario **
-----------  ------  -------  -------  --------------  -------------  --------------  --------------  --------------  --------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------  ------------------------  ------------------------  ----------------
group,       2016M,  2016LB,  2016UB,  2021M,          2021LB,        2021UB,         2026M,          2026LB,         2026UB,         2031M,           2031LB,          2031UB,          2051M,           2051LB, 

# Validation (Status Quo only)

BRFSS validation: 2016, 2017, 2021

prevalence of never smoker, former smoker, smoker, ecig/dual

for total pop, male, female, black

In [ ]:
# status quo
outputs = outputs_dirs[0]
collection_list = []

# for each arr, store a 2D array in the list
for f in sorted(glob(outputs + "/*.npy")):
    arr = np.load(f)
    arr = arr[[0, 1, 5,]] # get the years we are interested in
    arr = arr[:,:,:,:-1] # don't need dead people
    sums = np.sum(arr, axis=(1,2,3)) # total count for each year
    arr = arr / sums[:, np.newaxis, np.newaxis, np.newaxis] # get proportions
    collection_list.append(arr)

collection_list = np.array(collection_list)

In [ ]:
# sample, year, black, pov, state
print(collection_list.shape)

(62500, 3, 2, 2, 5)


In [ ]:
USpop = collection_list[:,:,:,:,:].sum(axis=(2,3))
Bpop = collection_list[:,:,1,:,:].sum(axis=2)
print(USpop.shape)
print(USpop.shape)

# combine smokers

USpop[:,:,2] += USpop[:,:,3]
Bpop[:,:,2] += Bpop[:,:,3]

USpop = USpop[:,:,[0,1,2,4]]
Bpop = Bpop[:,:,[0,1,2,4]]

print(USpop.shape)
print(USpop.shape)

(62500, 3, 5)
(62500, 3, 5)
(62500, 3, 4)
(62500, 3, 4)


In [ ]:
# black population denominator should not be whole population

if np.sum(Bpop, axis=2)[0,0] != 1:
    Bpop /= np.sum(Bpop, axis=2)[:,:,np.newaxis]

print(np.sum(Bpop, axis=2)[:5])

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [ ]:
for pop, name in zip([USpop, Bpop], ["US Population", "Black NH Population"]):

    mean_results = np.zeros_like(pop[0])
    upper_bound = np.zeros_like(pop[0])
    lower_bound = np.zeros_like(pop[0])

    for i in range(pop.shape[1]):
        for j in range(pop.shape[2]):
            mean = np.mean(pop[:,i,j])
            upper = np.percentile(pop[:,i,j], 97.5)
            lower = np.percentile(pop[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower
    
    # make a table

    mean_results = np.around(mean_results * 100, decimals=1)
    upper_bound = np.around(upper_bound * 100, decimals=1)
    lower_bound = np.around(lower_bound * 100, decimals=1)

    header = ["", "2016", "2017", "2021"]

    r1 = ["Never Smoker"]
    r2 = ["Former Smoker"]
    r3 = ["Cigarette Smoker"]
    r4 = ["Ecig/Dual"]

    rows = [r1, r2, r3, r4]

    for i, r in enumerate(rows):
        for m, ub, lb in zip(mean_results[:,i], upper_bound[:,i], lower_bound[:,i]):
            rows[i] += [f"{m}%, ({lb}, {ub})"]
    
    tab = [header] + rows
    print(" ")
    print(" ")
    print(f"Smoking Prevalences, Status Quo Scenario, {name}")
    printtab(tab)

    

 
 
Smoking Prevalences, Status Quo Scenario, US Population
                  2016                 2017                 2021
----------------  -------------------  -------------------  -------------------
Never Smoker      60.9%, (60.7, 61.1)  60.4%, (60.1, 60.6)  58.4%, (58.1, 58.8)
Former Smoker     20.3%, (20.1, 20.5)  22.2%, (21.9, 22.5)  28.1%, (27.7, 28.5)
Cigarette Smoker  15.1%, (15.1, 15.1)  14.5%, (14.3, 14.8)  11.7%, (11.4, 12.0)
Ecig/Dual         3.7%, (3.7, 3.7)     2.9%, (2.8, 3.1)     1.8%, (1.7, 2.0)
 
 
Smoking Prevalences, Status Quo Scenario, Black NH Population
                  2016                 2017                 2021
----------------  -------------------  -------------------  -------------------
Never Smoker      68.6%, (68.1, 69.1)  67.7%, (67.0, 68.4)  64.7%, (63.7, 65.7)
Former Smoker     11.2%, (10.7, 11.7)  13.4%, (12.7, 14.2)  19.4%, (18.4, 20.5)
Cigarette Smoker  18.3%, (18.1, 18.5)  17.3%, (16.7, 17.9)  14.9%, (14.1, 15.8)
Ecig/Dual         1.9%, (1.

# same but for CSV readability

In [ ]:
print("CSV values to be read into software")
print("All values in this table are percentages")
print("Status Quo scenario only, to be compared to BRFSS for validation")
print("In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)")
print("All numbers are rounded to 5 decimal places which should be more than we want for the final table")
for pop, name in zip([USpop, Bpop], ["US Population", "Black NH Population"]):

    mean_results = np.zeros_like(pop[0])
    upper_bound = np.zeros_like(pop[0])
    lower_bound = np.zeros_like(pop[0])

    for i in range(pop.shape[1]):
        for j in range(pop.shape[2]):
            mean = np.mean(pop[:,i,j])
            upper = np.percentile(pop[:,i,j], 97.5)
            lower = np.percentile(pop[:,i,j], 2.5)

            mean_results[i,j] = mean
            upper_bound[i,j] = upper
            lower_bound[i,j] = lower
    
    # make a table

    mean_results = np.around(mean_results * 100, decimals=5)
    upper_bound = np.around(upper_bound * 100, decimals=5)
    lower_bound = np.around(lower_bound * 100, decimals=5)

    header = ["2016", "2017", "2021"]
    new_header = ["group,"]
    for e in header:
        new_header.append(e + "M,")
        new_header.append(e + "LB,")
        new_header.append(e + "UB,") 

    r1 = ["NeverSmoker,"]
    r2 = ["FormerSmoker,"]
    r3 = ["CigaretteSmoker,"]
    r4 = ["Ecig/Dual,"]

    rows = [r1, r2, r3, r4]

    for i, r in enumerate(rows):
        for m, ub, lb in zip(mean_results[:,i], upper_bound[:,i], lower_bound[:,i]):
            rows[i] += [f"{m},", f"{lb},", f"{ub},"]
    
    tab = [new_header] + rows
    print(" ")
    print(" ")
    print(f"Smoking Prevalences, Status Quo Scenario, {name}")
    print(tabulate(tab))

CSV values to be read into software
All values in this table are percentages
Status Quo scenario only, to be compared to BRFSS for validation
In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)
All numbers are rounded to 5 decimal places which should be more than we want for the final table
 
 
Smoking Prevalences, Status Quo Scenario, US Population
----------------  --------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
group,            2016M,    2016LB,    2016UB,    2017M,     2017LB,    2017UB,    2021M,     2021LB,    2021UB,
NeverSmoker,      60.9061,  60.71185,  61.05806,  60.36026,  60.12619,  60.58145,  58.43383,  58.08637,  58.77357,
FormerSmoker,     20.2706,  20.11837,  20.478,    22.22535,  21.91023,  22.54255,  28.0728,   27.65194,  28.49849,
CigaretteSmoker,  15.1316,  15.1316,   15.1316,   14.51106,  14.26176,  14.76397,  11.67883,  11.36153,  11.99698,
Ecig/Dual,        3.6917,   3.66132, 

# NSDUH Validation

2016-2020

menthol cig use among smokers

US pop, black NH, In poverty


In [ ]:
# status quo
outputs = outputs_dirs[0]
collection_list = []

# for each arr, store a 2D array in the list
for f in sorted(glob(outputs + "/*.npy")):
    arr = np.load(f)
    arr = arr[[0, 1, 2, 3, 4]] # get the years we are interested in
    arr = arr[:,:,:,:-1] # don't need dead people
    collection_list.append(arr)

collection_list = np.array(collection_list)

In [ ]:
collection_list.shape

(62500, 5, 2, 2, 5)

In [ ]:
USprops = collection_list[:,:,:,:,2].sum(axis=(2,3)) / collection_list[:,:,:,:,2:4].sum(axis=(2,3,4))
print(USprops.shape)
Bprops = collection_list[:,:,1,:,2].sum(axis=2) / collection_list[:,:,1,:,2:4].sum(axis=(2,3))
print(Bprops.shape)
Pprops = collection_list[:,:,:,1,2].sum(axis=2) / collection_list[:,:,:,1,2:4].sum(axis=(2,3))
print(Pprops.shape)


(62500, 5)
(62500, 5)
(62500, 5)


In [ ]:
all_props = np.concatenate([
    USprops[:,np.newaxis,:],
    Bprops[:,np.newaxis,:],
    Pprops[:,np.newaxis,:],
], axis=1)
print(all_props.shape)

(62500, 3, 5)


In [ ]:
mean_results = np.zeros_like(all_props[0])
upper_bound = np.zeros_like(all_props[0])
lower_bound = np.zeros_like(all_props[0])

for i in range(all_props.shape[1]):
    for j in range(all_props.shape[2]):
        mean = np.mean(all_props[:,i,j])
        upper = np.percentile(all_props[:,i,j], 97.5)
        lower = np.percentile(all_props[:,i,j], 2.5)

        mean_results[i,j] = mean
        upper_bound[i,j] = upper
        lower_bound[i,j] = lower

# make a table

mean_results = np.around(mean_results * 100, decimals=1)
upper_bound = np.around(upper_bound * 100, decimals=1)
lower_bound = np.around(lower_bound * 100, decimals=1)

header = ["", "2016", "2017", "2018", "2019", "2020"]

r1 = ["US Population"]
r2 = ["Black NH"]
r3 = ["Poverty"]

rows = [r1, r2, r3,]

for i, r in enumerate(rows):
    for m, ub, lb in zip(mean_results[i], upper_bound[i], lower_bound[i]):
        rows[i] += [f"{m}%, ({lb} {ub})"]

tab = [header] + rows
print(" ")
print(" ")
print(f"Prevalence of Menthol Cigarette Smoking among Cigarette Smokers, Status Quo Scenario")
printtab(tab)

 
 
Prevalence of Menthol Cigarette Smoking among Cigarette Smokers, Status Quo Scenario
               2016                2017                2018                2019                2020
-------------  ------------------  ------------------  ------------------  ------------------  ------------------
US Population  38.0%, (37.9 38.2)  41.2%, (40.2 42.2)  44.6%, (43.4 45.7)  46.8%, (45.4 48.1)  48.6%, (47.1 50.0)
Black NH       82.8%, (82.4 83.4)  82.6%, (80.4 84.7)  84.3%, (81.9 86.7)  84.9%, (82.2 87.5)  85.4%, (82.6 88.1)
Poverty        45.0%, (44.8 45.2)  48.2%, (46.8 49.6)  51.3%, (49.7 53.0)  53.6%, (51.7 55.5)  55.5%, (53.4 57.5)


In [ ]:
print("CSV values to be read into software")
print("All values in this table are percentages")
print("Status Quo scenario only, to be compared to NSDUH for validation")
print("In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)")
print("All numbers are rounded to 5 decimal places which should be more than we want for the final table")
mean_results = np.zeros_like(all_props[0])
upper_bound = np.zeros_like(all_props[0])
lower_bound = np.zeros_like(all_props[0])

for i in range(all_props.shape[1]):
    for j in range(all_props.shape[2]):
        mean = np.mean(all_props[:,i,j])
        upper = np.percentile(all_props[:,i,j], 97.5)
        lower = np.percentile(all_props[:,i,j], 2.5)

        mean_results[i,j] = mean
        upper_bound[i,j] = upper
        lower_bound[i,j] = lower

# make a table

mean_results = np.around(mean_results * 100, decimals=5)
upper_bound = np.around(upper_bound * 100, decimals=5)
lower_bound = np.around(lower_bound * 100, decimals=5)

header = ["2016", "2017", "2018", "2019", "2020"]
new_header = ["group,"]
for e in header:
    new_header.append(e + "M,")
    new_header.append(e + "LB,")
    new_header.append(e + "UB,") 

r1 = ["USPopulation,"]
r2 = ["BlackPopulation,"]
r3 = ["PovPopulation,"]

rows = [r1, r2, r3,]

for i, r in enumerate(rows):
    for m, ub, lb in zip(mean_results[i], upper_bound[i], lower_bound[i]):
        rows[i] += [f"{m},", f"{lb},", f"{ub},"]

tab = [new_header] + rows
print(" ")
print(" ")
print(f"Prevalence of Menthol Cigarette Smoking among Cigarette Smokers, Status Quo Scenario")
print(tabulate(tab))

CSV values to be read into software
All values in this table are percentages
Status Quo scenario only, to be compared to NSDUH for validation
In column titles, M = mean, LB = lower bound, UB = upper bound (95% confidence intervals)
All numbers are rounded to 5 decimal places which should be more than we want for the final table
 
 
Prevalence of Menthol Cigarette Smoking among Cigarette Smokers, Status Quo Scenario
----------------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
group,            2016M,     2016LB,    2016UB,    2017M,     2017LB,    2017UB,    2018M,     2018LB,    2018UB,    2019M,     2019LB,    2019UB,    2020M,     2020LB,    2020UB,
USPopulation,     38.02958,  37.85917,  38.16062,  41.21162,  40.22402,  42.19851,  44.55508,  43.39774,  45.72271,  46.75773,  45.42542,  48.08927,  48.55062,  47.09928,  50.01096,
BlackPopulation,  82.81819,  82.40319